In [4]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import numpy as np
from datetime import datetime, timedelta

# Initialize Dash App
app = dash.Dash(__name__, external_stylesheets=["https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css"])
app.title = "Real-Time Trading Dashboard"

# Simulated Data
np.random.seed(42)
dates = [datetime.today() - timedelta(days=i) for i in range(30)]
dates.reverse()
prices = np.random.randint(100, 200, len(dates))
actions = np.random.choice([0, 1, 2], len(dates))  # 0: Hold, 1: Buy, 2: Sell

# Initialize variables for portfolio and signals
x_vals = []
y_vals = []
hold_signals = []
buy_signals = []
sell_signals = []
current_portfolio_value = 10000  # Starting portfolio value
monthly_portfolio_value = 10000
total_gains = 0
total_losses = 0
monthly_gains = 0
monthly_losses = 0

# Function to update plot data
def update_plot(t, price, action, x_vals, y_vals, hold_signals, buy_signals, sell_signals, 
                current_portfolio_value, monthly_portfolio_value, dates, 
                total_gains, total_losses, monthly_gains, monthly_losses):
    # Append data points
    x_vals.append(dates[t])
    y_vals.append(price)

    # Add signals based on action
    if action == 0:  # Hold
        hold_signals.append((dates[t], price))
    elif action == 1:  # Buy
        buy_signals.append((dates[t], price))
        total_gains += 0.05 * price  # Simulate profit
    elif action == 2:  # Sell
        sell_signals.append((dates[t], price))
        total_losses += 0.03 * price  # Simulate loss

    # Update portfolio values
    current_portfolio_value += 0.01 * price  # Simulate portfolio value change
    monthly_portfolio_value += 0.01 * price

    # Check for monthly marker
    is_month_start = t > 0 and dates[t].month != dates[t - 1].month
    marker_color = 'green' if monthly_gains > monthly_losses else 'red'

    return {
        "x_vals": x_vals,
        "y_vals": y_vals,
        "hold_signals": hold_signals,
        "buy_signals": buy_signals,
        "sell_signals": sell_signals,
        "portfolio_info": {
            "current_portfolio_value": current_portfolio_value,
            "monthly_portfolio_value": monthly_portfolio_value,
            "total_gains": total_gains,
            "total_losses": total_losses,
            "monthly_gains": monthly_gains,
            "monthly_losses": monthly_losses,
        },
        "is_month_start": is_month_start,
        "marker_color": marker_color,
        "date_marker": dates[t] if is_month_start else None,
    }

# Layout
app.layout = html.Div([
    html.H1("Real-Time Trading Performance", className="text-center my-3"),
    dcc.Graph(id="trading-graph"),
    html.Div([
        html.Div([
            html.H4("Current Portfolio Value:", className="mb-0"),
            html.P(id="current-portfolio", className="text-primary"),
        ], className="col"),
        html.Div([
            html.H4("Monthly Portfolio Value:", className="mb-0"),
            html.P(id="monthly-portfolio", className="text-success"),
        ], className="col"),
        html.Div([
            html.H4("Total Profit:", className="mb-0"),
            html.P(id="total-profit", className="text-success"),
        ], className="col"),
        html.Div([
            html.H4("Total Loss:", className="mb-0"),
            html.P(id="total-loss", className="text-danger"),
        ], className="col"),
    ], className="row g-3 mt-3"),
    dcc.Interval(id="update-interval", interval=1000, n_intervals=0)
])

# Callbacks
@app.callback(
    [Output("trading-graph", "figure"),
     Output("current-portfolio", "children"),
     Output("monthly-portfolio", "children"),
     Output("total-profit", "children"),
     Output("total-loss", "children")],
    [Input("update-interval", "n_intervals")]
)
def update_dashboard(n_intervals):
    # Fetch the latest data from the queue
    try:
        updated_data = data_queue.get_nowait()  # Get the latest data
    except queue.Empty:
        return dash.no_update  # No new data, keep the current state

    # Build Plotly figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=updated_data["x_vals"], y=updated_data["y_vals"], mode="lines+markers", name="Stock Price"))
    for signals, label, color in zip(
        [updated_data["hold_signals"], updated_data["buy_signals"], updated_data["sell_signals"]],
        ["Hold", "Buy", "Sell"],
        ["yellow", "green", "red"]
    ):
        if signals:
            signal_dates, signal_prices = zip(*signals)
            fig.add_trace(go.Scatter(x=signal_dates, y=signal_prices, mode="markers", name=label, marker=dict(color=color)))

    # Add monthly marker
    if updated_data["is_month_start"]:
        fig.add_vline(x=updated_data["date_marker"], line_color=updated_data["marker_color"], line_dash="dash")

    fig.update_layout(
        title="Real-Time Trading Performance",
        xaxis_title="Date",
        yaxis_title="Value ($)",
        template="plotly_dark"
    )

    portfolio_info = updated_data["portfolio_info"]

    return (
        fig,
        f"${portfolio_info['current_portfolio_value']:,.2f}",
        f"${portfolio_info['monthly_portfolio_value']:,.2f}",
        f"${portfolio_info['total_gains']:,.2f}",
        f"${portfolio_info['total_losses']:,.2f}"
    )


# Run Server
if __name__ == "__main__":
    app.run_server(debug=True)